## CS431/631 Data Intensive Distributed Analytics
### Winter 2023 - Assignment 3
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Chris Binoi Verghese
* **ID:** 21092999

---
#### Overview
For this assignment, you will be using Python and Spark to perform some graph analysis, using a graph of the Gnutella server network.   In this graph, each node represents a server, and each (directed) edge represents a connection between servers in Gnutella's peer-to-peer network.  The input file for this assignment, `p2p-Gnutella08-adj.txt`, represents the graph as an adjacency list.   Each server (node) is identified by a unique number, and each line in the file gives the adjacency list for a single server.
For example, this line:
> 91	243	1923	2194

gives the adjacency list for server `91`.   It indicates that there are edges from server `91` to servers `243`, `1923`, and `2194`.    According to the Stanford Network Analysis Project, which collected these data, [the graph includes 6301 servers and 20777 edges](http://snap.stanford.edu/data/p2p-Gnutella08.html).

Run the following block to install Spark and download the input file.

In [1]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.4.2/spark-3.4.2-bin-hadoop3.tgz
!tar xzf spark-3.4.2-bin-hadoop3.tgz
!pip install -q findspark
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/p2p-Gnutella08-adj.txt

and then create a `SparkContext`:

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.2-bin-hadoop3"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

---
## Important
### Watch Your Types
###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions.
###### This means: Do not change the type of value returned, do not add or remove parameters.
### No Top-Level Expressions
###### Do **NOT** write code outside of functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.  Better still, create new cells for your tests, then you do not need to remember to delete anything
### Dummy Values
###### The function prototypes all return a default value, labelled as a dummy.  This is so that the cell is runnable even if you do not do a question.  You should replace this value with the correct value.
### New Cells
###### The best way to test your code is to create a NEW code cell from which to call your functions.  We will **ONLY** run the code cells that **WE** provided.  So do not put any helper functions in these cells, unless the helper is **ONLY** for use with your tests.  Any helpers needed by your solutions themselves must go in the original code cell.  (All cells will be put into a single .py file, so you do not need to copy and paste a helper from one question to the next)

---
#### Question 1  (6/24 marks):

To get warmed up, you should first write Spark code to confirm or determine some basic properties of the Gnutella graph.  Write code in the provided functions that will return answers to the following questions, as specified in each function's documentation:
- How many nodes and edges are there in the graph?  (This should confirm the numbers given above.)
- How many nodes of each outdegree are there? That is, how many nodes have no outgoing edges, how many have one outgoing edge, how many have two outgoing edges, and so on?
- How many nodes of each indegree are there?

You should use Spark to answer these questions.   Do *not* load the entire graph into your Python driver program.

In [ ]:
def num_nodes_edges():
    """Returns a tuple (num_nodes, num_edges)"""
    #### Your code for Question 1.1 should go here
    #RDD which holds the node and its adjacency list
    adj_list = sc.textFile('p2p-Gnutella08-adj.txt')\
                  .map(lambda x: x.split('\t'))
    #Number of nodes is the number of adjacency lists
    nodes = adj_list.count()
    #The edges are calculated by taking the length-1 for each list
    edges = adj_list.map(lambda x:len(x)-1 if len(x)!=1 else 0)\
                    .reduce(lambda x,y: x+y)
    return (nodes,edges)


def out_counts():
    """Returns a dictionary where the keys are the outdegrees, and the
    values are the number of nodes of the corresponding outdegree """
    #### Your code for Question 1.2 should go here
    #Outdegree dictionary got by taking length of list of nodes with same outdegree
    out_deg = sc.textFile('p2p-Gnutella08-adj.txt')\
                  .map(lambda x: x.split('\t'))\
                  .map(lambda x: (len(x) - 1, x[0]))\
                  .groupByKey().mapValues(len).sortByKey()
    return dict(out_deg.collect())



def in_counts():
    """Returns a dictionary where the keys are the indegrees, and the
    values are the number of nodes of the corresponding indegree """
    #### Your code for Question 1.3 should go here

    #Function to map each server with (node, val)
    def in_deg_map(x):
        #Add server with no indegree
        ins = [(x[0],0)]
        if len(x)>1:
          for i in x[1:]:
            #Add node with 1 indegree
            ins.append((i,1))
        return ins

    #Each indegree count is calculated and used as key while the value is length of list
    nodes = sc.textFile('p2p-Gnutella08-adj.txt')\
              .map(lambda x: x.split('\t'))\
              .flatMap(in_deg_map).reduceByKey(lambda x,y: x+y)\
              .map(lambda x: (x[1],x[0]))\
              .groupByKey().mapValues(len)
    return dict(sorted(nodes.collect()))


print(num_nodes_edges())
print(out_counts())
print(in_counts())

(6301, 20777)
{0: 3836, 1: 294, 2: 28, 3: 16, 4: 28, 5: 107, 6: 10, 7: 9, 8: 44, 9: 372, 10: 1531, 11: 1, 12: 3, 13: 2, 14: 2, 17: 4, 18: 2, 19: 1, 22: 1, 24: 1, 25: 1, 28: 1, 29: 1, 31: 1, 34: 1, 41: 1, 46: 1, 47: 1, 48: 1}
{0: 80, 1: 2452, 2: 1287, 3: 868, 4: 559, 5: 333, 6: 227, 7: 144, 8: 76, 9: 70, 10: 37, 11: 29, 12: 23, 13: 19, 14: 13, 15: 8, 16: 1, 18: 2, 19: 2, 20: 4, 21: 2, 22: 1, 23: 1, 25: 1, 27: 1, 30: 1, 31: 2, 32: 2, 33: 1, 35: 1, 38: 1, 41: 1, 44: 1, 47: 2, 49: 1, 50: 1, 51: 1, 52: 1, 54: 1, 55: 1, 56: 2, 57: 1, 59: 1, 60: 3, 61: 1, 62: 2, 63: 1, 66: 2, 67: 3, 69: 2, 70: 3, 71: 3, 72: 2, 73: 2, 74: 1, 77: 2, 78: 1, 79: 1, 81: 4, 82: 1, 83: 1, 85: 1, 86: 1, 87: 1, 91: 1}


---
Your main objective for this assignment is to perform *single source personalized page rank* over the Gnutella graph.  To get started, you should make sure that you have a clear understanding of ordinary (i.e., non-personalized) page rank.  Read the description of page rank in Section 5.3 of [the course textbook](https://lintool.github.io/MapReduceAlgorithms/index.html).   Personalized page rank is like ordinary page rank except:
- One node in the graph is designated as the *source* node. Personalized page rank is performed with respect to that source node.
- Personalized page rank is initialized by assigning all probability mass to the source node, and none to the other nodes. In contrast, ordinary page rank is initialized by giving all nodes the same probability mass.
- Whenever personalized page rank makes a random jump, it jumps back to the source node. In contrast, ordinary page rank may jump to any node.
- In personalized page rank, all probability mass lost dangling nodes is put back into the source nodes.  In ordinary page rank, lost mass is distributed evenly over all nodes.

#### Question 2  (10/24 marks):

Your task is to write a Spark program to perform personalized page rank over the Gnutella graph for a specified number of iterations, and of course a specific node. The function you will implement takes three input values:
- source node id (a non-negative integer)
- iteration count (a positive integer)
- random jump factor value (a float between 0 and 1) - This is 1-B as introduced in the lecture.

The function should perform personalized page rank, with respect to the specified source node, over the Gnutella graph, for the specified number of iterations, using Spark.
The output of your function should be a list of the 10 nodes with the highest personalized page rank with respect to the given source. For each of the 10 nodes, return the node's id and page rank value as a tuple. The list returned by the function should therefore look something like this: `[(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]`

In [ ]:
def personalized_page_rank(source_node_id, num_iterations, jump_factor):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""
    # your solution to Question 2 here

    #Function which returns the node's page rank and impact on source node rank
    def pow_iter(x):
        out_nodes = x[1][0]
        node_rank = x[1][1]
        updated = [(source_node_id,node_rank*jump_factor)]
        out_deg = len(out_nodes)
        #If it is dead end, send rank to source to not sink rank
        if out_deg == 0:
          return [(source_node_id,node_rank)]
        #Calculate and append updated rank for each node
        for j in out_nodes:
          updated.append((int(j),node_rank * (1- jump_factor)/out_deg))
        return updated

    #RDD of each node and its out nodes
    adj_list = sc.textFile('p2p-Gnutella08-adj.txt')\
                 .map(lambda x: x.split('\t'))\
                 .map(lambda x: (int(x[0]),x[1:])).cache()
    #An initial page rank where only the source has rank
    page_rank = adj_list.map(lambda x: (x[0],0) if int(x[0]) != source_node_id else (x[0],1))
    #Loop to iterate for convergence of rank
    for i in range(num_iterations):
      #Combine previous page rank with adjacency list
      rank=adj_list.join(page_rank)
      #Get the updated page ranks
      page_rank = rank.flatMap(pow_iter).reduceByKey(lambda x,y: x+y)
      #print(page_rank.sortBy(lambda x: x[1],ascending=False).take(10))
      #print(page_rank.map(lambda pair: pair[1]).reduce(lambda x,y: x+y))
    return page_rank.sortBy(lambda x: x[1],ascending=False).take(10)

personalized_page_rank(0,10,0.15)


[(0, 0.3674273310683574),
 (9, 0.03286603326766931),
 (5, 0.03278391109520568),
 (7, 0.03269468512330566),
 (4, 0.03258901692214278),
 (8, 0.03196526429736632),
 (3, 0.03195278515151265),
 (10, 0.031415879346148326),
 (2, 0.031234246955940237),
 (1, 0.031233626242360167)]

---
#### Question 3  (4/24 marks):

For the previous question, you implemented personalized page rank that ran for a specified number of iterations.  However, it is also common to write iterative algorithms that run until some specified termination condition is reached.
For example, for page rank, suppose the $p_i(x)$ represents the probability mass assigned to node $x$ after the $i$th iteration of the algorithm.  ($p_0(x)$ is the initial probability mass of node $x$.)   We define the change of $x$'s probability mass on the $i$th iteration as $\lvert p_i(x)-p_{i-1}(x) \rvert$.   Then, we can iterate personalized page rank until the maximum (over all nodes) change is less than a specified threshold, i.e, until all nodes' page ranks have converged.

For this question, modify your personalized page rank implementation from Question 2 so that it iterates until the
maximum node change is less than $\frac{0.5}{N}$, where $N$ represents the number of nodes in the graph.
This version of the function should take only two inputs: the source node id and the random jump factor.

In [6]:
def personalized_page_rank_stopping_criterion(source_node_id, jump_factor):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""
    # your solution to Question 3 here

    #Function which returns the node's page rank and impact on source node rank
    def pow_iter(x):
        out_nodes = x[1][0]
        node_rank = x[1][1]
        updated = [(source_node_id,node_rank*jump_factor)]
        out_deg = len(out_nodes)
        #If it is dead end, send rank to source to not sink rank
        if out_deg == 0:
          return [(source_node_id,node_rank)]
        #Calculate and append updated rank for each node
        for j in out_nodes:
          updated.append((int(j),node_rank * (1- jump_factor)/out_deg))
        return updated


    adj_list = sc.textFile('p2p-Gnutella08-adj.txt')
    #Threshold
    prob = 0.5 / adj_list.count()
    #RDD of each node and its out nodes
    adj_list = adj_list.map(lambda x: x.split('\t'))\
                 .map(lambda x: (int(x[0]),x[1:])).cache()
    #An initial page rank where only the source has rank
    page_rank = adj_list.map(lambda x: (x[0],0) if int(x[0]) != source_node_id else (x[0],1))
    #Loop to iterate for convergence of rank
    while(True):
      #Combine previous page rank with adjacency list
      rank=adj_list.join(page_rank)
      #Get the updated page ranks
      page_rank = rank.flatMap(pow_iter).reduceByKey(lambda x,y: x+y).cache()
      #Get the max change in rank
      maximum = rank.join(page_rank).map(lambda x: abs(x[1][0][1]- x[1][1])).reduce(max)
      #If maximum change in rank is below the set threshold convergence is met
      if ( maximum <= prob ):
          break
    return page_rank.sortBy(lambda x: x[1],ascending=False).take(10)


personalized_page_rank_stopping_criterion(0,0.15)

[(0, 0.3674273310683574),
 (9, 0.03286603326766931),
 (5, 0.03278391109520568),
 (7, 0.03269468512330566),
 (4, 0.03258901692214278),
 (8, 0.03196526429736632),
 (3, 0.03195278515151265),
 (10, 0.031415879346148326),
 (2, 0.031234246955940237),
 (1, 0.031233626242360167)]

---
#### Question 4  (4/24 marks):

Spark provides the ability to *cache* RDDs.   This is useful when an RDD will be used more than once.   Instead of computing the same RDD multiple times, it can be computed once, cached, and then re-used from the cache.   Read about caching in the Spark textbook, or in the [persistence section of the Spark RDD programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence).   Caching can be particularly effective for iterative Spark applications, like those you are writing for this assignment.

For this question, go back to the code that you wrote to answer Question 3, and add caching.   (Caching will not affect the functionality of your code, i.e., what it computes.   It should only affect performance.)   Don't worry about different persistence levels.   Just use `cache()`, which caches RDDs at the default persistence level.

In addition to adding `cache()` calls in your code, use the text cell below to briefly explain how you decided which RDDs to cache.

If you were not able to finish Question 3, add caching annotations to your solution for Question 2 instead.

IMPORTANT: Because the files are small, the OS will retain the intermediate shuffle files so you won't actually see any improvement in you running times! (This means you have to think about what to cache rather than trying various places to trial and error it).

---
#### Your answer to Question 4:

I decided to cache the RDD which contains the node and its respective adjacency list called 'adj_list' as well as the newly calculated page rank values from each iteration which is stored in the RDD 'page_rank'.

The reasons behind deciding to cache adj_list was because it is computed only once after which its value is used in every iteration of the page rank algorithm. As such caching it will improve performance as it ensures that the data is kept in memory, reducing the overhead of reading the file and computing the adjacency list in each iteration.

The RDD 'page_rank' is also cached to improve improve performance. It is not reused to the same extent as adj_list however since page_rank's value are used more than once and over multiple iterations, caching it ensures that the intermediate results are kept in memory, avoiding recomputation of the entire RDD in each iteration. This is the reason why the RDD 'rank' containing the previous page rank is not cached, i.e. it is used only in one iteration at a time.



---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.